# Dashboards: How to Use

Dashboards are powerful tools for data analysis and visualization, so our JupyterHub has [Dash Plotly library](https://dash.plotly.com/) pre-installed with some additional features to provide dashboards available for users.

Here are very basic examples how to start your dashbord in JupyterHub environment.

## Dash installation

In your notebook or terminal, install `dash`:

In [ ]:
!pip install "dash<3.0.0"

## Simple dashboard

Let's create a "dummy" dashboard with no data in it:

In [ ]:
import os
from dash import Dash, html

In [ ]:
PORT = 20000
SRV_PREFIX = os.getenv('JUPYTERHUB_SERVICE_PREFIX', '/')
print('service prefix:', SRV_PREFIX)
BASE_URL = f'https://{os.getenv("JUPYTERHUB_HOST_NAME")}'
print('base url:', BASE_URL)

In [ ]:
# Initialize application

app = Dash(requests_pathname_prefix=f'{SRV_PREFIX}proxy/{PORT}/')
app.layout = html.Div(
    [
        html.H1('Minimal dashboard, only text in it.')
    ]
)

Now our dummy dashboard will be available at arbitary `port` (you better set port number from `10000` to `64000`), you will need to click URL of the dashboard provided:

In [ ]:
app.run(
    jupyter_mode='external',  
    jupyter_server_url=BASE_URL, 
    port=PORT
)

## Dashboard with diagram

Now will add a diagram to our dashboard:

In [ ]:
from dash import dcc

In [ ]:
app = Dash(requests_pathname_prefix=f'{SRV_PREFIX}proxy/{PORT}/')
app.layout = html.Div(children=[
    html.H1(children='Dash Example with Bar Plot'),
    html.Div(children='Dash: A web application framework for Python.'),
    dcc.Graph(
        id='example-bar-plot',
        figure={
            'data': [
                {
                    'x': ['A', 'B', 'C'], 
                    'y': [4, 1, 2], 
                    'type': 'bar', 
                    'name': 'Blue'
                },
                {
                    'x': ['A', 'B', 'C'], 
                    'y': [2, 4, 5], 
                    'type': 'bar', 
                    'name': 'Orange'
                },
            ],
            'layout': {
                'title': 'Dash Data Visualization'
            }
        }
    )
])

In [ ]:
app.run(
    jupyter_mode='external',  
    jupyter_server_url=BASE_URL, 
    port=PORT
)

## Advanced dashboard example

And finally, rather comlicated example of dashboard below:

In [ ]:
import plotly.express as px
from dash.dependencies import Input, Output

In [ ]:
# Load Data
df = px.data.tips()

# Build App
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(
    requests_pathname_prefix=f'{SRV_PREFIX}proxy/{PORT}/',
    external_stylesheets=external_stylesheets
)
app.layout = html.Div([
    html.H1('JupyterDash Demo'),
    dcc.Graph(id='graph'),
    html.Label([
        'colorscale',
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])

# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input('colorscale-dropdown', 'value')]
)

def update_figure(colorscale):
    return px.scatter(
        df, x='total_bill', y='tip', color='size',
        color_continuous_scale=colorscale,
        render_mode='webgl', title='Tips'
    )    

In [ ]:
app.run(
    jupyter_mode='external',  
    jupyter_server_url=BASE_URL, 
    port=PORT
)